In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
with open('./data/full.json') as f:
    data = json.load(f)

In [3]:
data[0]

{'Avtorji': None,
 'Datacija': 'zadnja četrtina 18. stol., 19. stol., prva polovica 20. stol., prva četrtina 21. stol., 2012',
 'ESD': 29884.0,
 'LokacijaOp': 'Muzej na prostem je urejen na južnem obrobju Šentruperta na Dolenjskem.',
 'Podrocja': 'etnologija, krajinska arhitektura, tehniška zgodovina',
 'Sinonimi': None,
 'Varstvo': 'spomenik lokalnega pomena',
 'Zavod': 'ZVKD Novo mesto',
 'Zvrst': 'stavbe',
 'description': 'Muzej na prostem sestavljajo skupina 17 kozolcev različnih tipov in dve enostavni sušilni napravi (belokranjska ostrv in ribniški kozouček). Postavitev iz 2012 prikazuje genezo kozolca na Slovenskem in raznolikost kozolcev v Mirnski dolini.',
 'keywords': 'muzej na prostem, kozolec na psa, kozolec s plaščem, enojni kozolec, nizki kozolec, toplar, vzporedni kozolec',
 'latitude': 45.9748324693,
 'longitude': 15.0914351743,
 'name': 'Muzej na prostem',
 'region': 'ŠENTRUPERT',
 'type': 'profana stavbna dediščina',
 'address': None,
 'city': None,
 'postal_code': Non

In [4]:
seed_coords = 46.329168, 14.232350

In [5]:
coords = np.empty((len(data), 2), dtype=float)

for idx, entry in enumerate(data):
    coords[idx] = [entry['latitude'], entry['longitude']]
    
coords

array([[45.97483247, 15.09143517],
       [46.55781501, 15.64419927],
       [46.18971074, 13.57812696],
       ...,
       [45.9579502 , 15.4952236 ],
       [46.3594701 , 15.1142968 ],
       [46.2390931 , 14.3553071 ]])

In [6]:
len(coords)

2198

In [7]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [8]:
distances = haversine(seed_coords[0], seed_coords[1], coords[:, 0], coords[:, 1])
distances

array([102.78529268, 158.80089261,  74.24061575, ..., 145.89073306,
        98.06055781,  16.75645814])

In [9]:
max_km_distance = 10
sd = max_km_distance / 2
probabilities = stats.norm.pdf(distances, loc=0, scale=sd)
probabilities /= np.sum(probabilities)

In [10]:
idx = np.random.choice(list(range(len(coords))), size=5, p=probabilities)
data[idx[0]]

{'Avtorji': None,
 'Datacija': 'tretja četrtina 19. stol., 1870, prelom 19. stol. in 20. stol., 1894, 1910',
 'ESD': 27644.0,
 'LokacijaOp': 'Bodlajev vigenjc stoji v jugozahodnem delu Krope imenovanem Zgornji Konec, ob desnem bregu potoka Kroparice.',
 'Podrocja': 'tehniška zgodovina',
 'Sinonimi': None,
 'Varstvo': 'dediščina',
 'Zavod': 'ZVKD Kranj',
 'Zvrst': 'stavbe',
 'description': 'Pritlična iz kamna zidana dvocelična stavba z dvokapnicama je kovačnica za ročno kovanje žebljev, sezidana leta 1870 na mestu lesenega vigenjca, ki je tega leta pogorel. Po letu 1894 jo kupi zadruga za žebljarsko obrt. Leta 1910 delo v njej opustij',
 'keywords': 'kovačija',
 'latitude': 46.2889619359,
 'longitude': 14.199442276,
 'name': 'Bodlajev vigenjc',
 'region': 'RADOVLJICA',
 'type': 'profana stavbna dediščina',
 'address': None,
 'city': None,
 'postal_code': None}

## Iterative recommender

In [11]:
current_coords = seed_coords

In [12]:
# Keep a list of locations that have not yet been visited
location_ids = list(range(len(coords)))

In [13]:
path = []

In [14]:
max_km_distance = 10
sd = max_km_distance / 2

In [15]:
for i in range(5):
    distances = haversine(current_coords[0], current_coords[1], coords[:, 0], coords[:, 1])
    
    probabilities = stats.norm.pdf(distances, loc=0, scale=sd)
    # Zero out already visited locations so the same place is not recommended twice
    probabilities[path] = 0
    probabilities /= np.sum(probabilities)
    
    idx = np.random.choice(location_ids, p=probabilities)
    
    path.append(idx)
    entry = data[idx]
    current_coords = entry['latitude'], entry['longitude']
    
path

[1584, 2048, 1613, 1599, 1606]

In [16]:
', '.join(data[idx]['name'] for idx in path)

'Kapusova graščina, Freska na hiši Rodine 9, Ruševine gradu Pusti grad, Graščina, Šivčeva hiša'